# BCG Case Study: 

**Author:** Ingrid Cadu<br>
**Data Source:** FORAGE.COM/BCG/<br>
**Last Update:** Apr, 05, 2022<br>

**Scenario**<br>
...<br>
## "How..."

## Imports

In [1]:
#Library

import datetime as dt
import numpy as np
import pandas as pd
import gmplot
import matplotlib.pyplot as plt
%matplotlib inline
import folium
import plotly as plt
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from pandas_profiling import ProfileReport

In [2]:
#Client Data as 'df_client' and price data as 'df_price'

df_client = pd.read_csv("client_data.csv")
df_price = pd.read_csv("price_data.csv")

## Data Exploration

In [3]:
#df_client dataframe

df_client.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,t,0.0,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,f,0.0,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,f,0.0,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0


In [4]:
#df_price dataframe

df_price.head(2)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0


### Quick Report

#### df_client

In [5]:
#Basic Info

df_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              14606 non-null  object 
 1   channel_sales                   14606 non-null  object 
 2   cons_12m                        14606 non-null  int64  
 3   cons_gas_12m                    14606 non-null  int64  
 4   cons_last_month                 14606 non-null  int64  
 5   date_activ                      14606 non-null  object 
 6   date_end                        14606 non-null  object 
 7   date_modif_prod                 14606 non-null  object 
 8   date_renewal                    14606 non-null  object 
 9   forecast_cons_12m               14606 non-null  float64
 10  forecast_cons_year              14606 non-null  int64  
 11  forecast_discount_energy        14606 non-null  float64
 12  forecast_meter_rent_12m         

In [6]:
#NANs

df_client.isna().sum()

id                                0
channel_sales                     0
cons_12m                          0
cons_gas_12m                      0
cons_last_month                   0
date_activ                        0
date_end                          0
date_modif_prod                   0
date_renewal                      0
forecast_cons_12m                 0
forecast_cons_year                0
forecast_discount_energy          0
forecast_meter_rent_12m           0
forecast_price_energy_off_peak    0
forecast_price_energy_peak        0
forecast_price_pow_off_peak       0
has_gas                           0
imp_cons                          0
margin_gross_pow_ele              0
margin_net_pow_ele                0
nb_prod_act                       0
net_margin                        0
num_years_antig                   0
origin_up                         0
pow_max                           0
churn                             0
dtype: int64

In [7]:
#Duplicated?

df_client.duplicated().sum()

0

In [8]:
#Unique categorical values - has gas?

round(df_client.has_gas.value_counts()/len(df_client)*100,0)

f    82.0
t    18.0
Name: has_gas, dtype: float64

In [9]:
#Unique categorical values - number of active products

round(df_client.nb_prod_act.value_counts()/len(df_client)*100,2)

1     78.26
2     16.74
3      3.58
4      1.03
5      0.21
9      0.08
6      0.05
8      0.03
10     0.01
32     0.01
Name: nb_prod_act, dtype: float64

In [10]:
#Unique categorical values - churn #just to machine learning interests

round(df_client.churn.value_counts()/len(df_client)*100,0)

0    90.0
1    10.0
Name: churn, dtype: float64

In [11]:
#Stats

df_client.describe()

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,pow_max,churn
count,1.460600e+04,1.460600e+04,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000
mean,1.592203e+05,2.809238e+04,16090.269752,1868.614880,1399.762906,0.966726,63.086871,0.137283,0.050491,43.130056,152.786896,24.565121,24.562517,1.292346,189.264522,4.997809,18.135136,0.097152
std,5.734653e+05,1.629731e+05,64364.196422,2387.571531,3247.786255,5.108289,66.165783,0.024623,0.049037,4.485988,341.369366,20.231172,20.230280,0.709774,311.798130,1.611749,13.534743,0.296175
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,3.300000,0.000000
25%,5.674750e+03,0.000000e+00,0.000000,494.995000,0.000000,0.000000,16.180000,0.116340,0.000000,40.606701,0.000000,14.280000,14.280000,1.000000,50.712500,4.000000,12.500000,0.000000
50%,1.411550e+04,0.000000e+00,792.500000,1112.875000,314.000000,0.000000,18.795000,0.143166,0.084138,44.311378,37.395000,21.640000,21.640000,1.000000,112.530000,5.000000,13.856000,0.000000
75%,4.076375e+04,0.000000e+00,3383.000000,2401.790000,1745.750000,0.000000,131.030000,0.146348,0.098837,44.311378,193.980000,29.880000,29.880000,1.000000,243.097500,6.000000,19.172500,0.000000
max,6.207104e+06,4.154590e+06,771203.000000,82902.830000,175375.000000,30.000000,599.310000,0.273963,0.195975,59.266378,15042.790000,374.640000,374.640000,32.000000,24570.650000,13.000000,320.000000,1.000000


#### df_price

In [12]:
#basic info

df_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193002 entries, 0 to 193001
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  193002 non-null  object 
 1   price_date          193002 non-null  object 
 2   price_off_peak_var  193002 non-null  float64
 3   price_peak_var      193002 non-null  float64
 4   price_mid_peak_var  193002 non-null  float64
 5   price_off_peak_fix  193002 non-null  float64
 6   price_peak_fix      193002 non-null  float64
 7   price_mid_peak_fix  193002 non-null  float64
dtypes: float64(6), object(2)
memory usage: 11.8+ MB


In [13]:
#NAN

df_price.isna().sum()

id                    0
price_date            0
price_off_peak_var    0
price_peak_var        0
price_mid_peak_var    0
price_off_peak_fix    0
price_peak_fix        0
price_mid_peak_fix    0
dtype: int64

In [14]:
#duplicated?

df_price.id.duplicated().sum()

176906

In [15]:
#Stats

df_price.describe()

,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
count,193002.000000,193002.000000,193002.000000,193002.000000,193002.000000,193002.000000
mean,0.141027,0.054630,0.030496,43.334477,10.622875,6.409984
std,0.025032,0.049924,0.036298,5.410297,12.841895,7.773592
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.125976,0.000000,0.000000,40.728885,0.000000,0.000000
50%,0.146033,0.085483,0.000000,44.266930,0.000000,0.000000
75%,0.151635,0.101673,0.072558,44.444710,24.339581,16.226389
max,0.280700,0.229788,0.114102,59.444710,36.490692,17.458221


### Full Report - df_client

In [16]:
#Using Pandas
#------------------------

#profile = ProfileReport(df_client, title="Churn Analysis Report")

#profile.to_widgets()

#profile.to_file('04052022_Churn_Analysis_Report.html')

## Data Preparation

In [17]:
#Exchanging data type of df_client and df_cprice


cols = ['date_activ','date_end','date_modif_prod','date_renewal']

for x in df_client[cols]:
    df_client[x] = pd.to_datetime(df_client[x], format='%Y-%m-%d')
    

df_price['price_date'] = pd.to_datetime(df_price['price_date'], format='%Y-%m-%d')

## Data Analysis

In [18]:
#Filtering the year = 2016
#

df16 = df_client[df_client["date_renewal"].dt.date > dt.date(2016,1,1)]
df16['label'] = np.where(df16['churn']==1,'churn','customer')

In [19]:
#sum margin_net_pow_ele

mnetp_sum = round(df16['margin_net_pow_ele'].sum(),2)
mnetp_sum

32000.61

In [20]:
#sum net_margin

netm_sum = round(df16["net_margin"].sum(),2)
netm_sum

185496.23

In [21]:
#avg consume over 12m

cons12 = round(df16["cons_12m"].mean(),2)
cons12

161745.82

In [22]:
#avg consume gas over 12m

cons_gas12 = round(df16["cons_gas_12m"].mean(),2)
cons_gas12

42787.09

In [ ]:
#avg consume last month

cons_lmonth = round(df16["cons_last_month"].mean(),2)
cons_lmonth

In [119]:
#how many rewewed contracts?
#rew15 = df_client[df_client["date_renewal"].dt.date > dt.date(2015,1,1)]
#len(rew15) 2015-:> 14303 and 2016 -:> 903
#round(len(rew15)/len(df_client)*100,2) 
#97.93% = 91.75%(2015) + 6.18%(2016) & 8.25 (< 2015)
#how many renewed the contract in 2016 and churn at same year?
#round(len(df16)/len(df_client)*100,2) #6.18% renewed

#pies = {'Year': ['< 2015','2015','2016'], 'value': [8.25, 91.75, 6.18]} #donut chart
label = ['customer','churn']
value = round(df16['churn'].value_counts()/len(df16)*100,1) #95 churn people of 903 clients (7)

hovertemp = "<b>Class: </b> %{label} <br>"
hovertemp += "<b>Total: </b> %{value: .2f}%"

fig = go.Figure(go.Pie(labels=label, textfont_size= 18, values=value, hole=.8, hovertemplate=hovertemp))
fig.update_traces(textfont_family='Arial', marker_colors=['#245274','#85263c'], selector=dict(type='pie'), showlegend=True)
fig.update_layout(width=300, height=400, margin=dict(l=5,r=5,b=15,t=5,pad=4),
                   paper_bgcolor="white",
                   plot_bgcolor='white',
                   title_font_family="Arial",
                   title_font_color="#444",
                 showlegend=False)
fig.show()

#85263c - customer color
#245274 - churn color

In [116]:
#how many renewed contract had gas? #donut chart (8)

labels = ['No', 'Yes']
value = round(df16.has_gas.value_counts()/len(df16)*100,2)
hovertemp = "<b>Gas: </b> %{label} <br>"
hovertemp += "<b>Total: </b> %{value: .2f}%"

fig = go.Figure(go.Pie(labels=labels, 
                       values=value,
                       hovertemplate=hovertemp,
                       hole=.8))
fig.update_traces(textfont_family='Arial', textfont_size=18, 
                  marker_colors=['#85263c','#245274'], selector=dict(type='pie'), 
                  showlegend=True)
fig.update_layout(width=300, height=400, margin=dict(l=5,r=5,b=15,t=5,pad=4),
                   paper_bgcolor="white",
                   plot_bgcolor='white',
                   title_font_family="Arial",
                   title_font_color="#444",
                 showlegend=False)

fig.show()

In [177]:
#how many years old were the churn people? #area chart (10)

df = df_client.groupby(['num_years_antig']).sum()

fig = go.Figure(go.Bar(x=df.index, y=df['churn']))

# Customize aspect
fig.update_layout(width=900, height=400, 
                  hovermode="x unified",
                  showlegend=False, 
                  margin=dict(l=5,r=5,b=5,t=5,pad=4),
                  paper_bgcolor="white",
                  plot_bgcolor='white',
                  font_family="Arial",
                  font_color="#444",
                  title_font_family="Arial",
                  title_font_color="#444")
fig.update_traces(hovertemplate="<extra></extra><b>Year(s): </b> %{x}<br> <b>Total: </b> %{y}", 
                  marker_color='#33374b', 
                  selector=dict(type='bar'))
fig.update_xaxes(type='category', showgrid=False, showticklabels=False)
fig.update_yaxes(showticklabels=False, showgrid=False)


In [41]:
#df_price grouping people by mean (5)

dfp = df_price.groupby(['id']).mean()

#merging df_price and dF_client to know the price average over the year CAN FILTER BY CHURN ;D change the color!

df_dfp = pd.merge(df16, dfp, on=["id"], how="left")
df_dfp2 = df_dfp.loc[:,["price_off_peak_var","price_peak_var","price_mid_peak_var","price_off_peak_fix","price_peak_fix","price_mid_peak_fix", "churn", "label"]].copy()

data1 = df_dfp[df_dfp['label']=='churn']

fig = px.violin(data1, y=["price_off_peak_var","price_peak_var","price_mid_peak_var"], 
                labels={'value':'', 'variable':''})
fig.update_layout(width=800, height=400, 
                  showlegend=False,margin=dict(l=5,r=5,b=5,t=5,pad=4),
                   paper_bgcolor="white",
                   plot_bgcolor='white',
                   font_family="Arial",
                   font_color="#444",
                   title_font_family="Arial",
                   title_font_color="#444")
fig.update_traces( marker_color="#33374b", selector=dict(type='violin'))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showticklabels=False, showgrid=False)

fig.show()

In [43]:
#Fixed price - graph (6)

fig = fig = px.violin(data1, y=["price_off_peak_fix","price_peak_fix","price_mid_peak_fix"], labels={'value':'', 'variable':''})
fig.update_layout(width=800, height=400, 
                  showlegend=False,margin=dict(l=5,r=5,b=5,t=5,pad=4),
                   paper_bgcolor="white",
                   plot_bgcolor='white',
                   font_family="Arial",
                   font_color="#444",
                   title_font_family="Arial",
                   title_font_color="#444")
fig.update_traces( marker_color="#33374b", selector=dict(type='violin'))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showticklabels=False, showgrid=False)
fig.show()

In [181]:
#how many renewed contract had active products? CAN FILTER BY CHURN! #bar chart (9)

data2 = df16[df16['label']=='customer']

x = round(data2.nb_prod_act.value_counts()/len(data2)*100,2)

fig = go.Figure(go.Bar(x=x.values[:3],y=x.index[:3],orientation='h'))
fig.update_layout(width=500, height=400, xaxis_showgrid=False, yaxis_showgrid=False)

fig.update_layout(width=500, height=400, 
                  hovermode="y unified",
                  showlegend=False,
                  margin=dict(l=5,r=5,b=5,t=5,pad=4),
                   paper_bgcolor="white",
                   plot_bgcolor='white',
                   font_family="Arial",
                   font_color="#444",
                   title_font_family="Arial",
                   title_font_color="#444")
fig.update_traces(hovertemplate="<extra></extra><b>Qtd. Product(s): </b> %{y}<br> <b>Total: </b> %{x}%",
                  marker_color='#33374b', selector=dict(type='bar'))
fig.update_xaxes(showgrid=False, showticklabels=False)
fig.update_yaxes(showticklabels=False, showgrid=False)

fig.show()
#33374b

### df_client data merged to save

In [29]:
df_merged = pd.merge(df_client, dfp, on=["id"], how="left")
df_merged.isna().sum()

#new dataframe to machine learning
new_data = df_merged.loc[:,["id","churn","price_off_peak_var","price_peak_var","price_mid_peak_var","price_off_peak_fix","price_peak_fix","price_mid_peak_fix"]].copy()
new_data.head()

,id,churn,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,24011ae4ebbe3035111d65fa7c15bc57,1,0.124787,0.100749,0.066530,40.942265,22.352010,14.901340
1,d29c2c54acc38ff3c0614d0a653813dd,0,0.149609,0.007124,0.000000,44.311375,0.000000,0.000000
2,764c75f661154dac3a6c254cd082ea7d,0,0.170512,0.088421,0.000000,44.385450,0.000000,0.000000
3,bba03439a292a1e166f80264c16191cb,0,0.151210,0.000000,0.000000,44.400265,0.000000,0.000000
4,149d57cf92fc41cf94415803a877cb4b,0,0.124174,0.103638,0.072865,40.688156,24.412893,16.275263


## Others

### Save files as *.csv

In [30]:
#new_data.to_csv("churn_peak.csv")

In [ ]:
df16.to_csv("data.csv", index=False)

In [31]:
#colors
#background= #e2e0d9
#fonts #444